In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd

from src.tickers import TickerDownloader, TickerManager

from src.helpers_allocation import (
    calculate_each_year_allocation,
)

from src.pdf.generate_report import generate_report

In [12]:
tickers_downloader = TickerDownloader()

df_tickers = tickers_downloader.download_tickers_sp500(20)["Close"]
df_tickers = TickerManager(df_tickers)

sp500_market = tickers_downloader.download_tickers_data(["^GSPC"], x_years=20, keep_null=True)["Close"]

[*********************100%***********************]  1 of 1 completed


In [13]:
money_value, df_money = calculate_each_year_allocation(df_tickers, 4)

/usr/local/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:262: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:262: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:262: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:262: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:262: UserWarning: max_s

In [14]:
generate_report(df_money, sp500_market)